In [ ]:
import config
import numpy as np
import pandas as pd
import load_utils

# Loading Data

In [ ]:
geojsons = load_utils.load_geojsons() 

In [ ]:
start_date = '2013-11-01'
n_days     = 1

In [ ]:
provinces_df, countries_df = load_utils.load_provinces_and_countries(start_date, n_days)

In [ ]:
import pycountry
from phonenumbers import COUNTRY_CODE_TO_REGION_CODE

def create_code_to_country(countries_df):
    code_to_country = {}
    country_codes = set(countries_df['countrycode'])
    for country_code in country_codes:
        
        if country_code == 0:
            code_to_country[country_code] = 'Domestic'
        else:
            try:
                region_codes = COUNTRY_CODE_TO_REGION_CODE[country_code]
                for region_code in region_codes:
                    country_name = pycountry.countries.get(alpha_2=region_code).name
                    if country_code not in code_to_country:
                        code_to_country[country_code]  = [country_name]
                    else:
                        code_to_country[country_code] += [country_name]
            except KeyError:
                pass
    return code_to_country

code_to_country = create_code_to_country(countries_df)

In [ ]:
provinces = sorted(list(set(provinces_df['provinceName'])))

countries = []
for value in code_to_country.values():
    for country in value:
        countries.append(country) 
countries = sorted(countries)

In [ ]:
province_columns = list(provinces_df.columns[2:])
country_columns  = list(countries_df.columns[2:])

In [ ]:
def create_columns_names(provinces_df, countries_df,
                         provinces, countries):
    
    
    def create_column_names(columns, regions):
        column_names = []
        for column in columns:
            for region in regions:
                column_names.append('{}_{}'.format(column, region))
        return column_names
           
    column_names  = create_column_names(provinces_df.columns[2:], provinces)
    column_names += create_column_names(countries_df.columns[2:], countries)
    return column_names

column_names = create_columns_names(provinces_df, countries_df, provinces, countries)
        

In [ ]:
n_cells     = 10000
n_hours     = 24 * n_days

In [ ]:
time_indices = pd.concat([pd.DataFrame(np.repeat(provinces_df.index.drop_duplicates().values, n_cells), columns=['datetime'])]).reset_index(drop=True)
cell_indices = pd.concat([pd.DataFrame(np.arange(n_cells), columns=['CellID'])] * n_hours).reset_index(drop=True)
df_index     = pd.MultiIndex.from_tuples(list(zip(list(time_indices['datetime']), list(cell_indices['CellID']))), names=['datetime', 'CellID'])

In [ ]:
# Keep it as an array for now in order to populate it faster
df = np.zeros((n_hours*n_cells, len(column_names)))
row_indices_map    = {key: value for value, key in enumerate(df_index.values)}
column_indices_map = {key: value for value, key in enumerate(column_names)}

In [ ]:
import sys
import progressbar
def populate_df(df, row_indices_map, column_indices_map,
                regions_df, code_to_country=None):
    
    if 'provinceName' in regions_df:
        source = 'provinces_df'
    else:
        source = 'countries_df'
    
    print('Populating with data from "{}"'.format(source))
    sys.stdout.flush()
    
    bar = progressbar.ProgressBar(max_value=len(regions_df))
    column_names = regions_df.columns[2:]
    
    for i, row in enumerate(regions_df.to_records()):
        try:
            row = list(row)
            time_index, cell_index, region = row[:3]
            if 'countrycode' in regions_df:
                regions = code_to_country[region]
            else:
                regions = [region]
            values = row[3:]
            
            for column_name, value in zip(column_names, values):
                for region in regions:
                    df_column_name = '{}_{}'.format(column_name, region)
                    row_index    = row_indices_map[(time_index, cell_index)]
                    column_index = column_indices_map[df_column_name]
                    df[row_index, column_index] = value
                
        except KeyError as e:
            pass
        bar.update(i)

In [ ]:
populate_df(df, row_indices_map, column_indices_map, provinces_df)
populate_df(df, row_indices_map, column_indices_map, countries_df, code_to_country)

In [ ]:
# Convert to DataFrame
df = pd.DataFrame(df, columns=column_names, index=df_index)

# VISUALIZATION

In [ ]:
from bokeh.palettes import Viridis256 as palette
from bokeh.models import LinearColorMapper as mapper

color_mapper = mapper(palette=palette, low=0, high=255)
TOOLS = "pan,wheel_zoom,reset,hover,save"

In [ ]:
from bokeh.plotting import figure
from bokeh.models import HoverTool

from bokeh.models import (
 GMapPlot, GMapOptions, Patches, Range1d,
)

def grid_plot(source, title, color_mapper, tools):
    p = figure(
       title=title, tools=tools,
       x_axis_location=None, y_axis_location=None
    )
    p.grid.grid_line_color = None

    p.patches('lon', 'lat', source=source,
             fill_color={'field': 'calls', 'transform': color_mapper},
             fill_alpha=0.5, line_color=None, line_width=1)

    hover = p.select_one(HoverTool)
    hover.point_policy = "follow_mouse"
    hover.tooltips = [
       ("Name", "@names"),
       ("Calls)", "@calls"),
       ("(Lat, Lon)", "(@center_lat, @center_lon)"),
    ]
    
    return p

def gmap_plot(source, title, color_mapper, center_coors, api_key):
    map_options = GMapOptions(lat=center_coors['lat'], lng=center_coors['lon'], 
                              map_type="hybrid", zoom=11, scale_control=True)
    p = GMapPlot(x_range=Range1d(), y_range=Range1d(), map_options=map_options)
    p.grid.grid_line_color = None
    p.title.text = title
    p.api_key = api_key

    patches = Patches(xs='lon', ys='lat',
                 fill_color={'field': 'calls', 'transform': color_mapper},
                 fill_alpha=0.5, line_color=None, line_width=1)
    p.add_glyph(source, patches)
    
    return p

In [ ]:
# To use bokeh.io.export_png you need selenium ("conda install -c bokeh selenium" or "pip install selenium")
# pip install pillow==4.0.0
# To use bokeh.io.export_png you need pillow ("conda install pillow" or "pip install pillow")
# PhantomJS is not present in PATH. Try "conda install phantomjs

from bokeh.io import show, output_file, export_png
from bokeh.models import ColumnDataSource
from bokeh.layouts import column

def plot_milan(df, geojsons, color_mapper, periods=1, gmap=True):
    output_file("milan.html")

    lon = [[coors[0] for coors in cell["geometry"]["coordinates"][0]] for cell in geojsons['grid']['features']]
    lat = [[coors[1] for coors in cell["geometry"]["coordinates"][0]] for cell in geojsons['grid']['features']]
    names = [cell["id"] for cell in geojsons['grid']['features']]

    date_range = pd.date_range(df.index[0][0], periods=periods, freq='1H')

    plot_data = {}
    for i, date in enumerate(date_range):

        hour_df = df.loc[pd.IndexSlice[str(date)]]

        traffic_per_cell = hour_df.sum(axis=1)
        traffic_per_cell = np.log(traffic_per_cell) # Smooth the data
        plot_data[date] = traffic_per_cell

    maximum = np.array(list(plot_data.values())).max()

    plots = []
    bar = progressbar.ProgressBar(max_value=len(plot_data))
    for i, (date, traffic_per_cell) in enumerate(plot_data.items()):

        calls = traffic_per_cell.copy()
        calls /= maximum
        calls *= 255
        calls = calls.replace([np.inf, -np.inf], np.nan).fillna(0)
        calls = calls.astype(int)

        source = ColumnDataSource(data=dict(
            lon=lon,
            lat=lat,
            names=names,
            calls=calls,
            center_lon=[np.mean(x) for x in lon],
            center_lat=[np.mean(x) for x in lat],
        ))
        
        title = 'Cell Phone Usage in the City of Milan on {} at {}'.format(
                    date.strftime('%Y-%m-%d'), date.strftime('%H-%M-%S'))

        if not gmap:
            p = grid_plot(source, title, color_mapper, TOOLS)
        else:
            center_coors = {
                'lat': np.asarray(lat).mean(),
                'lon': np.asarray(lon).mean()}
            p = gmap_plot(source, title, color_mapper, center_coors, config.gmaps_api_key)
            
#         export_png(p, filename='{}.png'.format(str(date)))

        plots.append(p)
        bar.update(i)
        
    return plots

In [ ]:
plots = plot_milan(df, geojsons, color_mapper, n_hours)
show(column(*plots))

In [ ]:
# Fix the provinces whose names are not matching between DataFrame and geojson
replacement_dict = {
    'MASSA CARRARA': 'MASSA-CARRARA'
}

for province in geojsons['provinces']['features']:
    province_name = province['properties']['PROVINCIA'].upper()
    if province_name in replacement_dict:
        province['properties']['PROVINCIA'] = replacement_dict[province_name]

In [ ]:
def plot_italy(df, geojsons, color_mapper):
    output_file("italy.html")

    lon = [[coors[0] for coors in province["geometry"]["coordinates"][0][0]] for province in geojsons['provinces']['features']]
    lat = [[coors[1] for coors in province["geometry"]["coordinates"][0][0]] for province in  geojsons['provinces']['features']]
    names = [province["properties"]["PROVINCIA"].upper() for province in geojsons['provinces']['features']]
    calls = [df[df.filter(like=province).columns].sum().sum() for province in names]
    
    calls = np.log(calls) # Smoothen
    calls /= np.asarray(calls).max() # Normalize
    calls *= 255 # Scale to pallete
    
    source = ColumnDataSource(data=dict(
       lon=lon,
       lat=lat,
       names=names,
       calls=calls,
       center_lon=[np.mean(x) for x in lon],
       center_lat=[np.mean(x) for x in lat],
    ))

    p = figure(
       title="Italian Provinces by Number of Calls with Milan", tools=TOOLS,
       x_axis_location=None, y_axis_location=None
    )
    p.grid.grid_line_color = None

    p.patches('lon', 'lat', source=source,
             fill_color={'field': 'calls', 'transform': color_mapper},
             fill_alpha=0.7, line_color="white", line_width=0.5)

    hover = p.select_one(HoverTool)
    hover.point_policy = "follow_mouse"
    hover.tooltips = [
       ("Name", "@names"),
       ("Calls)", "@calls"),
       ("(Lat, Lon)", "(@center_lat, @center_lon)"),
    ]
    return p

In [ ]:
p_italy = plot_italy(df, geojsons, color_mapper)
show(p_italy)

In [ ]:
def plot_world(df, geojsons, color_mapper):
    output_file("world.html")
    
    lon = []
    lat = []
    names = []
    calls = []
    
    for country in geojsons['countries']['features']:  
        if country["id"] == '-99': # Norther Cyprus
            country["id"] = "CYP"
        elif country["id"] == 'CS-KM': # Kosovo
            country["id"] = "SRB"
        name = pycountry.countries.get(alpha_3=country["id"]).name
        
        if len(df.filter(like=name).columns) == 0:
            print(name)

        geometry = country["geometry"]
        if geometry['type'] == 'Polygon':
            country_borders = [geometry["coordinates"][0]]

        elif geometry['type'] == 'MultiPolygon':
            country_borders = []
            for polygon in geometry["coordinates"]:
                country_borders += polygon
        else:
            raise ValueError('Unknown type of geojson')

        for polygon in country_borders:
            lon.append([])
            lat.append([])
            names.append(name)
            calls.append(df[df.filter(like=name).columns].sum().sum())
            for coors in polygon:
                lon[-1].append(coors[0])
                lat[-1].append(coors[1])
                
    calls = np.log(calls) # Smoothen
    calls /= np.asarray(calls).max() # Normalize
    calls *= 255 # Scale to pallete
    calls = calls.tolist()

    source = ColumnDataSource(data=dict(
       lon=lon,
       lat=lat,
       names=names,
       calls=calls,
       center_lon=[np.mean(x) for x in lon],
       center_lat=[np.mean(x) for x in lat],
    ))

    p = figure(
       title="Countries by Number of Calls with Milan", tools=TOOLS,
       x_axis_location=None, y_axis_location=None,
       plot_width=1500, plot_height=600
    )
#     p.grid.grid_line_color = None

    p.patches('lon', 'lat', source=source,
             fill_color={'field': 'calls', 'transform': color_mapper},
             fill_alpha=0.7, line_color="white", line_width=0.5)

    hover = p.select_one(HoverTool)
    hover.point_policy = "follow_mouse"
    hover.tooltips = [
       ("Name", "@names"),
       ("Calls)", "@calls"),
       ("(Lat, Lon)", "(@center_lat, @center_lon)"),
    ]
    
    return p

In [ ]:
p_world = plot_world(df, geojsons, color_mapper)
show(p_world)

In [ ]:
countries

# ANOMALY DETECTION

In [ ]:
# Copy df to data in order to keep the data untouched
data = df.copy()

In [ ]:
# Normalize the data
data -= data.mean()
data /= data.std()
data = data.fillna(0.0)

In [ ]:
# Add Random Anomalies
def add_anomalies(data, percentage=0.01):
    n_anomalies = int(percentage * len(data)) # 1% of data
    data = data.append(pd.DataFrame(np.zeros((n_anomalies, data.shape[1])), columns=data.columns))
    for index in range(len(data) - n_anomalies, len(data)):
        # Random connections
        n_random_indices = np.random.randint(4, 10)
        random_indices = np.random.randint(0, data.shape[1],  n_random_indices)
        data.iloc[index, random_indices] = np.random.uniform(-1.0, 1.0, size=n_random_indices)
    return data, n_anomalies

data, n_anomalies = add_anomalies(data)

In [ ]:
# Add CellIDs
data['CellID'] = 0
data.loc[data.index.values[:-n_anomalies], 'CellID'] = np.array([cell_index for _, cell_index in df.index.values])
data.loc[data.index.values[-n_anomalies:], 'CellID'] = np.random.randint(0, n_cells, n_anomalies)

In [ ]:
# Add Labels
data['anomaly'] = 0
data.loc[data.index.values[-n_anomalies:], 'anomaly'] = 1

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2)

In [ ]:
def batches(data, batch_size=32678):
    
    n_batches = len(data)//batch_size
    print('Data consists of {} batches'.format(n_batches))
    sys.stdout.flush()
    
    bar = progressbar.ProgressBar(max_value=n_batches)
    for batch_index in range(0, n_batches):
        
        bar.update(batch_index)
        
        batch = data.iloc[batch_index*batch_size:(batch_index+1)*batch_size]
        
        one_hot = np.zeros((batch_size, n_cells))
        for i in range(len(one_hot)):
            one_hot[i, batch['CellID'].values[i]] = 1
        one_hot = pd.DataFrame(one_hot, columns=['CellID_'+str(i) for i in range(n_cells)], index=batch.index)
        batch = batch.join(one_hot)
        
        yield batch

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

print('Training the Model')
for batch in batches(train_data):
    clf.fit(batch.drop(['CellID', 'anomaly'], axis=1), batch['anomaly'])

In [ ]:
from sklearn.metrics import confusion_matrix

print('Evaluating the Model')
y_true = np.array([])
y_pred = np.array([])
for batch in batches(test_data, batch_size=4096):
    y_true = np.append(y_true, batch['anomaly'].values)
    y_pred = np.append(y_pred, clf.predict(batch.drop(['CellID', 'anomaly'], axis=1)))

print(confusion_matrix(y_true, y_pred))

In [ ]:
indices = np.where(y_pred == 1)
index = indices[0][0]
anomaly_example = test_data.iloc[index]
cell_id = int(anomaly_example['CellID'])
print(anomaly_example[anomaly_example > 0], end='\n\n')

In [ ]:
normal_examples = data.iloc[:-n_anomalies][data.iloc[:-n_anomalies]['CellID'] == cell_id]
for i, (index, example) in enumerate(normal_examples.iterrows()):
    print(example[example > 0], end='\n\n')
    
    if i == 5:
        break